In [1]:
print("test")

test


In [92]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [93]:
def query_Azure(sql):
    
    server = 'azr-gbk-db01.database.windows.net' 
    database = 'ELASTIC_MASTER' 
    username = 'jungsoo.lee' 
    password = '!jsl0814GBK' 
    AZURE = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    conn = pyodbc.connect(AZURE)
    df = pd.read_sql(sql, conn)

    return df

sql = f"""
--sql
SELECT
 MBR.branch_name
,MBP.address
,MBP.city
,MBP.state1
,MBP.zipcode
,MBR.region_reporting
FROM [BI_MASTER].[MASTER_BRANCH] MBR
LEFT JOIN [BI_MASTER].[Master_Business_Partners_Distinct_DC] MBP
ON MBR.sap_vcode = MBP.sap_vcode
WHERE MBR.active = 'Y'
ORDER BY region_reporting
;
"""
store_df = query_Azure(sql)

C:\Users\jungsoo.lee\AppData\Local\Temp\ipykernel_9468\1761368401.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [94]:
store_df

,branch_name,address,city,state1,zipcode,region_reporting
0,Union,29-02 UNION ST,FLUSHING,NY,11354,R1
1,Northern 156,156-40 NORTHERN BLVD,FLUSHING,NY,11354,R1
2,Great Neck,495 GREAT NECK RD.,GREAT NECK,NY,11021,R1
3,Williston Park,400 HILLSIDE AVE.,WILLISTON PARK,NY,11596,R1
4,Hartsdale,371 N. CENTRAL AVE,HARTSDALE,NY,10530,R1
...,...,...,...,...,...,...
75,Irvine3,3931 IRVINE BLVD.,IRVINE,CA,92602,R8
76,Brookline,1026-1028 BEACON ST,BROOKLINE,MA,02446,R9
77,Quincy,101 FALLS BLVD,QUINCY,MA,02169,R9
78,Cambridge,581 MASS AVE,CAMBRIDGE,MA,02139,R9


In [95]:
# # webscrape data 
# for store in store_df['placer_name']:
#     # create list of data for each store
#     df_lst = []
#     for t in time_periods:
#         # search store address and change time period

#         # export data
#         df = pd.read_csv()

#         # save data
#         df_lst.append(df)


In [96]:
store_lon_lat = pd.read_csv("G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Datasets\Ranked Venues - H Mart nationwide (Total).csv",skiprows=1)
store_lon_lat


,Rank,Name,Store ID,Total Visits,lat,lng
0,1,"H Mart / 3995 Alemany Blvd, San Francisco, CA,...",NaN,142109,37.710072,-122.467819
1,2,"H Mart / 1101 W Huntington Dr, Arcadia, CA, Un...",NaN,121439,34.131640,-118.065765
2,3,"H Mart / 2625 Old Denton Rd, Ste 200, Carrollt...",NaN,115034,32.984908,-96.912479
3,4,"H Mart / 1761 NJ-27, Edison, NJ, United States",NaN,114270,40.513621,-74.407937
4,5,"H Mart / 8911 Garden Grove Blvd, Garden Grove,...",NaN,96769,33.774851,-117.976491
...,...,...,...,...,...,...
70,71,"H Mart / 400 Hillside Avenue, Williston Park, ...",NaN,23271,40.754371,-73.656311
71,72,"H Mart / 156-40 Northern Blvd, Flushing, NY, U...",NaN,21917,40.762892,-73.808942
72,73,"H Mart / 16 E Golf Rd Unit E, Schaumburg, IL, ...",NaN,20925,42.051011,-88.077724
73,74,"H Mart / 141-40 Northern Blvd, Flushing, NY, U...",11.0,15649,40.764304,-73.824092


In [97]:
# define time periods to compare
today = datetime.now()
first = today.replace(day=1)

# most recent month, prior month, last year same month
last = (first + relativedelta(months=-2))
prior = (first + relativedelta(months=-3))
LY = (first + relativedelta(months=-2) + relativedelta(years=-1))

# a list of folder names (time periods)
tp_lst = []
for x in [last, prior, LY]:
    tp_lst.append("Favorite Places - H Mart, H Mart and 1 more - "
                   + x.strftime("%b"+" " +"%#d"+", "+"%Y") + " - "
                   + (x + relativedelta(day=31)).strftime("%b"+" " +"%#d"+", "+"%Y")
                   )

# a list of file names (stores)
store_lst = 'H Mart - ' + store_df['address'].replace('[^a-zA-Z\s]','',regex=True).apply(lambda x: x.strip()) + ", " + store_df['city'] + ", " + store_df['state1']

In [98]:
# load raw data of all store's favorite places in different time periods (total 79 * 3 df in one dictionary)
def load_df(tp_lst, store_lst):
    # create dataframe dictionary
    df_dict = {}

    root = "G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Datasets"
    
    # import csv and append to df_lst using file path
    for i in range(len(tp_lst)):
        store_df = []
        for j in range(len(store_lst)):
        # read csv files
            dir = f'{root}\{tp_lst[i]}\{store_lst[j]}.csv'
            df = pd.read_csv(dir)
            if store_lst[j] not in df_dict.keys():
                df_dict[store_lst[j]] = [df]
            else:
                df_dict[store_lst[j]].append(df)
    
    return df_dict

In [99]:
store_lon_lat[store_lon_lat['Name'].str.contains('Irvine|Carrollton|Little Ferry')]

,Rank,Name,Store ID,Total Visits,lat,lng
2,3,"H Mart / 2625 Old Denton Rd, Ste 200, Carrollt...",NaN,115034,32.984908,-96.912479
16,17,"H Mart / 260 Bergen Tpke, Little Ferry, NJ, Un...",38.0,77013,40.856226,-74.033270
24,25,"H Mart / 3825 Alton Pkwy, Irvine, CA, United S...",NaN,73590,33.683712,-117.813829
43,44,"H Mart / 3931 Irvine Blvd, CA, United States",NaN,54097,33.724200,-117.772319
47,48,"H Mart / 2600 Alton Pkwy, Irvine, CA, United S...",NaN,49003,33.689248,-117.834456


In [100]:
temp_location_dict = {'H Mart - BERGEN TPKE, LITTLE FERRY, NJ':[40.856226,-74.033270],
                      'H Mart - OLD DENTON RD, CARROLLTON, TX':[32.984908,-96.912479],
                      'H Mart - ALTON PKWY, IRVINE, CA':[33.683712,-117.813829]}

In [101]:
store_df[store_df['branch_name'].isin(['Irvine','Carrollton','Little Ferry'])]

,branch_name,address,city,state1,zipcode,region_reporting
17,Little Ferry,260 BERGEN TPKE,LITTLE FERRY,NJ,07643,R2
54,Carrollton,2625 OLD DENTON RD #200,CARROLLTON,TX,75007,R7
57,Irvine,2600 ALTON PKWY,IRVINE,CA,92606,R8


In [102]:
temp_store_lst = [store_lst[17],store_lst[54],store_lst[57]]
temp_tp_lst = tp_lst

In [103]:
temp_dict = load_df(temp_tp_lst, temp_store_lst)
temp_dict['H Mart - BERGEN TPKE, LITTLE FERRY, NJ'][0]

,ID,Name,Category Group,Category,Sub Category,Address,City,Location,Distance (Miles),# of Visitors,% of Visitors
0,60826b74b6097e189bb63fd4,Teterboro Landing,Shopping Centers,Community Shopping Centers,Shopping Center,2 Teterboro Landing Dr,Teterboro,"40.860944935779514, -74.06275328081634",1.6,23256,41.1%
1,a0a3d4863399d9e8b31a85f9,Westfield Garden State Plaza,Shopping Centers,Community Shopping Centers,Shopping Center,500 Garden State Plz,Paramus,"40.91690181412384, -74.07392942306461",4.7,15713,27.7%
2,802c7afce4eb8f41a5ec0110,Costco Wholesale,Superstores,Superstores,Warehouse Store,2 Teterboro Landing Dr,Teterboro,"40.86074968763845, -74.0644842614966",1.7,14059,24.8%
3,5a57100c750e6db7c67aa8b4,American Dream,Shopping Centers,Community Shopping Centers,Shopping Center,1 American Dream Wy,East Rutherford,"40.80857249450705, -74.0690370346953",3.8,9362,16.5%
4,b8ba85519a3973ae2702c740,Hackensack Plaza,Shopping Centers,Community Shopping Centers,Shopping Center,450 Hackensack Avenue,Hackensack,"40.90867278835986, -74.03022520250343",3.6,8953,15.8%
...,...,...,...,...,...,...,...,...,...,...,...
819,a8a4311f16a990f70b6c3225,Dollar Deal,Shops & Services,Discount & Dollar Stores,Discount Store,518 Kinderkamack Rd,River Edge,"40.92565964456048, -74.03384706966008",4.8,167,<0.5%
820,314656317ea173febbf6185b,Staples,Office Supplies,Office Supplies,Paper / Office Supplies Store,280 Route 17 Suite 197,East Rutherford,"40.82513366104003, -74.09587129840585",3.9,165,<0.5%
821,d8d94ee333ec4f3e14f47809,The Home Depot,Home Improvement,Home Improvement,Hardware Store,106 NJ-23,Riverdale,"40.99079193513926, -74.32702125001312",17.9,159,<0.5%
822,ed6ddcce8a5b207413459df0,Walmart,Superstores,Superstores,Big Box Store,300 Wootton St,Boonton,"40.90717657450498, -74.40410472312242",19.7,159,<0.5%


In [104]:
ltf = pd.read_csv('G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Datasets\H mart -  Bergen Tpke, Favorite Places Weekly Visit Share %.csv')#.set_index('Week Start Date')
ltf = ltf.iloc[:5,:][['Week Start Date','H Mart/ 321 Broad Ave','HanNam Mart/ 1475 Bergen Blvd', 'H Mart/ 60 State Route 17 North','99 Ranch Market/ 450 Hackensack Ave', 'ShopRite/ 500 S River St']].apply(lambda x: x*100)
ltf

,Week Start Date,H Mart/ 321 Broad Ave,HanNam Mart/ 1475 Bergen Blvd,H Mart/ 60 State Route 17 North,99 Ranch Market/ 450 Hackensack Ave,ShopRite/ 500 S River St
0,7/31/20237/31/20237/31/20237/31/20237/31/20237...,6.4,4.2,3.8,3.0,3.1
1,8/7/20238/7/20238/7/20238/7/20238/7/20238/7/20...,5.2,2.9,2.2,3.2,6.0
2,8/14/20238/14/20238/14/20238/14/20238/14/20238...,5.1,4.1,2.0,2.1,3.2
3,8/21/20238/21/20238/21/20238/21/20238/21/20238...,4.8,3.4,2.9,3.2,4.4
4,8/28/20238/28/20238/28/20238/28/20238/28/20238...,4.1,3.9,1.7,2.5,4.3


In [105]:
car = pd.read_csv('G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Datasets\H mart -  Old Denton Rd, Favorite Places Weekly Visit Share %.csv')#.set_index('Week Start Date')
car = car.iloc[:5,:][['Week Start Date','99 Ranch Market/ 2352 Old Denton Rd', 'H Mart/ 3320 K Ave','Hiep Thai Food Store/ 2430 E Pioneer Pkwy','H-E-B/ 6001 Preston Rd Suite 100','Mitsuwa Marketplace/ 100 Legacy Dr']].apply(lambda x: x*100)
car

,Week Start Date,99 Ranch Market/ 2352 Old Denton Rd,H Mart/ 3320 K Ave,Hiep Thai Food Store/ 2430 E Pioneer Pkwy,H-E-B/ 6001 Preston Rd Suite 100,Mitsuwa Marketplace/ 100 Legacy Dr
0,7/31/20237/31/20237/31/20237/31/20237/31/20237...,8.6,2.3,1.5,0.9,1.4
1,8/7/20238/7/20238/7/20238/7/20238/7/20238/7/20...,7.1,1.8,0.7,2.1,7.0
2,8/14/20238/14/20238/14/20238/14/20238/14/20238...,7.2,1.6,0.8,1.8,1.3
3,8/21/20238/21/20238/21/20238/21/20238/21/20238...,7.7,1.6,1.3,0.7,1.5
4,8/28/20238/28/20238/28/20238/28/20238/28/20238...,8.7,1.9,1.2,1.0,1.1


In [106]:
irv = pd.read_csv('G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Datasets\H mart -  Alton Pkwy, Favorite Places Weekly Visit Share %.csv')#.set_index('Week Start Date')
irv = irv.iloc[:5,:][['Week Start Date', 'H Mart/ 2600 Alton Pkwy','Mitsuwa Marketplace/ 665 Paularino Ave','Zion Market/ 4800 Irvine Blvd', 'Mitsuwa Marketplace/ 14230 Culver Dr','99 Ranch Market/ 15333 Culver Dr Suite #800']].apply(lambda x: x*100)
irv

,Week Start Date,H Mart/ 2600 Alton Pkwy,Mitsuwa Marketplace/ 665 Paularino Ave,Zion Market/ 4800 Irvine Blvd,Mitsuwa Marketplace/ 14230 Culver Dr,99 Ranch Market/ 15333 Culver Dr Suite #800
0,07/31/2307/31/2307/31/2307/31/2307/31/2307/31/...,1.9,2.4,2.9,2.4,3.4
1,08/07/2308/07/2308/07/2308/07/2308/07/2308/07/...,3.6,3.2,2.8,2.5,4.7
2,08/14/2308/14/2308/14/2308/14/2308/14/2308/14/...,3.3,3.0,2.3,2.1,2.8
3,08/21/2308/21/2308/21/2308/21/2308/21/2308/21/...,3.7,2.8,3.0,3.2,3.3
4,08/28/2308/28/2308/28/2308/28/2308/28/2308/28/...,3.2,2.3,2.5,2.4,3.4


In [107]:
temp_weekly_dict = {'H Mart - BERGEN TPKE, LITTLE FERRY, NJ':ltf,
                      'H Mart - OLD DENTON RD, CARROLLTON, TX': car,
                      'H Mart - ALTON PKWY, IRVINE, CA':irv}

In [108]:
# clean each dataframe in dictionary and return new dictionary with cleaned dataframes
def clean_dict(tp_lst, store_lst):
    # load all raw data in one dictionary
    temp_dict = load_df(tp_lst, store_lst)
    # create new dictionary to return
    cleaned_dict = {}
    for store in store_lst:
        df_lst = temp_dict[store]
        cleaned_df_lst = []
        for df in df_lst:
            # filter category
            new_df = df[df['Category']=='Groceries']
            # remove other hmart stores from the df
            # new_df = new_df[new_df['Name'].str.contains('H Mart')==False]
            # create Name/ Address column
            new_df['Name/ Address'] = new_df['Name']+' / '+new_df['Address'] +", "+new_df['City']
            # convert % to float from str
            new_df['% of Visitors'] = new_df['% of Visitors'].apply(lambda x : x.replace('%','')).astype(float)
            # get latitude and longitude
            new_df['Lat'] = df['Location'].str.split(',').str[0].astype(float)
            new_df['Lon'] = df['Location'].str.split(',').str[1].astype(float)
            # drop irrelevant columns and sort by %
            new_df = new_df[['Name/ Address','Lat','Lon','Distance (Miles)','# of Visitors','% of Visitors']]

            cleaned_df_lst.append(new_df)

        cleaned_dict[store] = cleaned_df_lst

    return cleaned_dict

# combine favorite places in three different time periods in one dataframe and return new dictionary with combined dataframes
def combine_dict(tp_lst, store_lst):
    # load the dictionary with cleaned dataframes
    cleaned_dict = clean_dict(tp_lst, store_lst)
    # create a new dictionary to return
    combined_dict = {}
    for store in store_lst:
        df_lst = cleaned_dict[store]
        # set base table as current time period favorite places
        current_top10 = df_lst[0].nlargest(10,'% of Visitors')
        # left join base table with prior and ly
        new_df = pd.merge(current_top10,df_lst[1],on='Name/ Address',how='left',suffixes=('',' prior'))
        new_df = pd.merge(new_df,df_lst[2],on='Name/ Address',how='left',suffixes=('',' LY'))
        # fill 0 if prior & ly data not available
        new_df['% of Visitors prior'] = new_df['% of Visitors prior'].fillna(0)
        new_df['% of Visitors LY'] = new_df['% of Visitors LY'].fillna(0)
        # create variance col's
        new_df['Variance prior'] = new_df['% of Visitors'] - new_df['% of Visitors prior']
        new_df['Variance LY'] = new_df['% of Visitors'] - new_df['% of Visitors LY']
        # create rank col
        new_df['Rank'] = new_df['% of Visitors'].rank(method='min',ascending=False).astype('int8')
        # drop irrelevant columns
        new_df = new_df[['Rank','Name/ Address','Lat','Lon','Distance (Miles)','# of Visitors','% of Visitors','% of Visitors prior','Variance prior','% of Visitors LY','Variance LY']]
        
        combined_dict[store] = new_df

    return combined_dict



In [121]:
def map_table(tp_lst, store_lst):
    combined_dict = combine_dict(tp_lst, store_lst)
    token = 'pk.eyJ1IjoianVuZ3Nvb2xlZTEyMyIsImEiOiJjbG5pemliaWcwdDc4MmxtanlsazFjNnphIn0.DuoiXA_AYMRMk1ZQjzS6ZA'
    for store in store_lst:
        fig = make_subplots(
            rows=2,
            cols=2,
            specs = [[{"type": "mapbox", "rowspan":2},{"type": "table"}]
                     ,[None, {"type":"scatter"}]],
            subplot_titles=("","Competitor List","Weekly Trend"),
            vertical_spacing=0.02,
            horizontal_spacing=0.08
            )
        # map visualization
        df = combined_dict[store]
        # add h mart marker
        fig.add_trace(go.Scattermapbox(
            lat=[temp_location_dict[store][0]],
            lon=[temp_location_dict[store][1]],
            mode='markers',
            marker = go.scattermapbox.Marker(
                    size=20,color='blue',
                    opacity=0.7),
            hovertext=[store],
            hoverinfo='text',
            showlegend=False
            ),row=1,col=1)

        # add competitors markers
        color_scale = [(0, 'orange'), (1,'red')]
        fig.add_trace(go.Scattermapbox(
                lat=df['Lat'],
                lon=df['Lon'],
                mode='markers+text',
                marker=go.scattermapbox.Marker(
                    size=df["% of Visitors"]*5,
                    color=df["% of Visitors"],
                    # autocolorscale=True,
                    colorscale=color_scale,
                    colorbar=dict(title='% of Visitors', x=0.02),
                    opacity=0.7
                ),
                text=list(df['Rank']),
                name='',
                hovertext=list(df['Name/ Address']),
                hoverinfo='text',
                showlegend=False
            ),row=1,col=1)
        # fig.add_trace(px.scatter_mapbox(df, 
        #                         lat="Lat", 
        #                         lon="Lon", 
        #                         #hover_name="Name/ Address", 
        #                         #hover_data=["Name/ Address","Distance (Miles)", "% of Visitors"],
        #                         color="% of Visitors",
        #                         color_continuous_scale=color_scale,
        #                         size="% of Visitors",
        #                         zoom=10, 
        #                         height=500,
        #                         width=800,
        #                         ),row=1,col=1)
        # fig.update_layout(mapbox_style="carto-positron", margin={"r":10,"t":40,"l":10,"b":10},title = f'{store}')
        # fig.show()

        # table visualization
        dfx = df[['Rank','Name/ Address','Distance (Miles)', '% of Visitors','Variance prior','Variance LY']].round(2)
        fig.add_trace(go.Table( columnwidth=[10,60,10,10,10,10],
                                header=dict(values=['Rank','Name/ Address','Distance (Miles)', '% of Visitors','Last Month','Last Year'],
                                            font_size=16),
                                cells=dict(values=[dfx[k].tolist() for k in dfx.columns],
                                           font_color=['darkslategray','darkslategray','darkslategray','darkslategray',["red" if x <= 0 else "green" if x >= 0 else 'darkslategray' for x in list(dfx['Variance prior'])],["red" if x <= 0 else "green" if x >= 0 else 'darkslategray' for x in list(dfx['Variance LY'])]],
                                           height=30)
            ),row=1,col=2)

        # stacked area chart       
        dfw = temp_weekly_dict[store]
        for competitor in dfw.columns[1:]:
            fig.add_trace(go.Scatter(
                name=competitor,
                x=['W1','W2','W3','W4','W5'],
                y=dfw[competitor],
                stackgroup='one',
                hovertext=list(dfw[competitor]),
                hoverinfo='text',
            ),row=2,col=2)

        # update layout   
        fig.update_annotations(font_size=18,yshift=20)
        fig.update_yaxes(title_text="% of Visitors", row=2, col=2)
        fig.update_xaxes(title_text="Week #", row=2, col=2)
        fig.update_layout(
                mapbox=dict(accesstoken=token,zoom=12,style='light',center={'lon':temp_location_dict[store][1],'lat':temp_location_dict[store][0]}),
                margin={"r":20,"t":100,"l":20,"b":10},
                title = f'{store}',
                title_font_size=24,
                width=2400,height=1000,
                legend=dict(y=0.4)
                )
        fig.write_html(f"G:\Shared drives\Grand BK - Corporate Strategy\jungsoo\Placer_AI\Etc\{store}.html") 
        fig.show()


In [122]:
map_table(temp_tp_lst, temp_store_lst)

C:\Users\jungsoo.lee\AppData\Local\Temp\ipykernel_9468\1819421996.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jungsoo.lee\AppData\Local\Temp\ipykernel_9468\1819421996.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jungsoo.lee\AppData\Local\Temp\ipykernel_9468\1819421996.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda